## COD API, Matches --> explore, format, clean & reshape

#### import stuff to work with

In [1]:
import asyncio
import os
import sys
import dotenv
from pprint import pprint
import datetime
from datetime import datetime, timezone, timedelta
import pandas as pd
import numpy as np
import pickle

import callofduty
from callofduty import Mode, Platform, Title, TimeFrame, GameType

In [2]:
# Using SSO
# We're storing our SSO token in an .env file stored locally to separate our config from code (w. python-dotenv). An.env-template file (with help to retrieve token) is provided for you to edit and populate the variable(s)
# callofduty.py client .Login() goes through all the authentification steps and initiate a session to access protected routes
# The client is asynchronous thus the 'await style'
from dotenv import load_dotenv
load_dotenv()
client = await callofduty.Login(sso=os.environ["SSO"])

In [3]:
# we're storing our string conversion module above current working (notebooks) directory
sys.path.insert(0, os.path.abspath('../wzkd'))
import labels
from labels import mode_labels

In [4]:
# This time we're adding additional methods in the Call of Duty .py client only as there is no need to modify the HTTP class that already contains the endpoint we want to use
# Import the Class we want to modify

import urllib.parse
from typing import List, Optional, Union

from callofduty.client import Client
from callofduty.http import HTTP
from callofduty.http import Request


# following additional methods to be added in callofduty.client.py Client Class
# see notebooks/cod_api_doc.ipnyb for details

async def GetMatches(
    self, platform, username: str, title: Title, mode: Mode, **kwargs
):

    limit: int = kwargs.get("limit", 20)
    startTimestamp: int = kwargs.get("startTimestamp", 0)
    endTimestamp: int = kwargs.get("endTimestamp", 0)

    data: dict = (
        await self.http.GetPlayerMatches(
            platform,
            username,
            title.value,
            mode.value,
            limit,
            startTimestamp,
            endTimestamp,
        )
    )["data"] # API res was filtered out here

    return data


async def GetMatchesDetailed(
    self, platform, username: str, title: Title, mode: Mode, **kwargs
):

    limit: int = kwargs.get("limit", 20)
    startTimestamp: int = kwargs.get("startTimestamp", 0)
    endTimestamp: int = kwargs.get("endTimestamp", 0)

    return (
        await self.http.GetPlayerMatchesDetailed(
            platform,
            username,
            title.value,
            mode.value,
            limit,
            startTimestamp,
            endTimestamp,
        )
    )["data"]['matches'] # API res was filtered out here


async def GetMatchesSummary(
    self, platform, username: str, title: Title, mode: Mode, **kwargs
):

    limit: int = kwargs.get("limit", 20)
    startTimestamp: int = kwargs.get("startTimestamp", 0)
    endTimestamp: int = kwargs.get("endTimestamp", 0)

    return (
        await self.http.GetPlayerMatchesDetailed(
            platform,
            username,
            title.value,
            mode.value,
            limit,
            startTimestamp,
            endTimestamp,
        )
    )["data"]['summary'] # API res was filtered out here


# add our modified methods into callofduty Client Class

Client.GetMatches = GetMatches
Client.GetMatchesDetailed = GetMatchesDetailed
Client.GetMatchesSummary = GetMatchesSummary

# raw_matches_datailed has indeed two keys : summary and matches (the one that is filtered out in the Client)
matches = await client.GetMatchesDetailed("battle", "AMADEVS#1689", Title.ModernWarfare, Mode.Warzone, limit=20)


In [5]:
# save file if you want to work in offline mode
#with open("matches.pkl", 'wb') as f:
#    pickle.dump(matches, f)

### Overview of returned stats

In [6]:
# load offline data
open_file = open("matches.pkl", "rb")
loaded_matches = pickle.load(open_file)
open_file.close()

In [7]:
pprint(loaded_matches[0], depth=2)

{'draw': False,
 'duration': 795000,
 'gameType': 'wz',
 'map': 'mp_don4',
 'matchID': '4779687662268956307',
 'mode': 'br_rumble_clash',
 'player': {'awards': {},
            'clantag': 'lkf :',
            'loadout': [...],
            'loadouts': [...],
            'rank': 54.0,
            'team': 'axis',
            'uno': '2621859779580650696',
            'username': 'gentil_renard'},
 'playerCount': 100,
 'playerStats': {'assists': 4.0,
                 'bonusXp': 0.0,
                 'challengeXp': 0.0,
                 'damageDone': 2925.0,
                 'damageTaken': 823.0,
                 'deaths': 8.0,
                 'distanceTraveled': 74418.586,
                 'executions': 0.0,
                 'gulagDeaths': 0.0,
                 'gulagKills': 0.0,
                 'headshots': 5.0,
                 'kdRatio': 1.5,
                 'kills': 12.0,
                 'longestStreak': 5.0,
                 'matchXp': 6616.0,
                 'medalXp': 680.0,
    

#### raw data

In [8]:
raw = pd.DataFrame(loaded_matches)
raw.head(5)

,utcStartSeconds,utcEndSeconds,map,mode,matchID,duration,playlistName,version,gameType,playerCount,playerStats,player,teamCount,rankedTeams,draw,privateMatch
0,1636049077,1636049872,mp_don4,br_rumble_clash,4779687662268956307,795000,None,1,wz,100,"{'kills': 12.0, 'medalXp': 680.0, 'matchXp': 6...","{'team': 'axis', 'rank': 54.0, 'awards': {}, '...",2,None,False,False
1,1635974372,1635975965,mp_don4,br_brquads,3670970687107920841,1593000,None,1,wz,152,"{'kills': 0.0, 'medalXp': 0.0, 'matchXp': 3220...","{'team': 'team_eighteen', 'rank': 54.0, 'award...",39,None,False,False
2,1635972898,1635974510,mp_don4,br_brquads,2776548585301277918,1612000,None,1,wz,152,"{'kills': 0.0, 'medalXp': 0.0, 'matchXp': 1010...","{'team': 'team_twenty_three', 'rank': 54.0, 'a...",38,None,False,False
3,1635971331,1635972934,mp_don4,br_brquads,12283805539552672972,1603000,None,1,wz,151,"{'kills': 2.0, 'medalXp': 20.0, 'objectiveTeam...","{'team': 'team_fourteen', 'rank': 54.0, 'award...",38,None,False,False
4,1635970345,1635971989,mp_don4,br_brquads,3648604903481562804,1644000,None,1,wz,152,"{'kills': 4.0, 'medalXp': 20.0, 'objectiveTeam...","{'team': 'team_eighteen', 'rank': 54.0, 'award...",40,None,False,False


In [9]:
raw.keys()

Index(['utcStartSeconds', 'utcEndSeconds', 'map', 'mode', 'matchID',
       'duration', 'playlistName', 'version', 'gameType', 'playerCount',
       'playerStats', 'player', 'teamCount', 'rankedTeams', 'draw',
       'privateMatch'],
      dtype='object')

#### playerStats col

In [10]:
player_stats = raw['playerStats'].apply(pd.Series)
player_stats.head(20)

,kills,medalXp,matchXp,scoreXp,wallBangs,score,totalXp,headshots,assists,challengeXp,...,objectiveTeamWiped,objectiveLastStandKill,objectiveBrDownEnemyCircle3,objectiveBrDownEnemyCircle1,objectiveMunitionsBoxTeammateUsed,objectiveBrDownEnemyCircle4,objectiveBrDownEnemyCircle2,objectiveReviver,objectiveTrophyDefense,objectiveDestroyedVehicleMedium
0,12.0,680.0,6616.0,1900.0,0.0,1875.0,9196.0,5.0,4.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.0,0.0,3220.0,1525.0,0.0,300.0,4745.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.0,0.0,10107.0,5040.0,0.0,2200.0,15147.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2.0,20.0,11096.0,5345.0,0.0,4325.0,16677.0,0.0,4.0,0.0,...,1.0,2.0,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN
4,4.0,20.0,6606.0,4025.0,0.0,2925.0,10666.0,1.0,0.0,0.0,...,1.0,2.0,NaN,4.0,3.0,NaN,NaN,NaN,NaN,NaN
5,3.0,40.0,10671.0,4195.0,0.0,2675.0,15661.0,0.0,3.0,0.0,...,NaN,1.0,NaN,NaN,NaN,2.0,1.0,6.0,NaN,NaN
6,0.0,0.0,3703.0,1650.0,0.0,1150.0,5353.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
7,0.0,0.0,4399.0,700.0,0.0,200.0,5099.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2.0,30.0,6383.0,2350.0,0.0,1000.0,8763.0,0.0,1.0,0.0,...,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
9,1.0,20.0,3404.0,900.0,0.0,800.0,4324.0,0.0,0.0,0.0,...,NaN,2.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
player_stats.keys()

Index(['kills', 'medalXp', 'matchXp', 'scoreXp', 'wallBangs', 'score',
       'totalXp', 'headshots', 'assists', 'challengeXp', 'rank',
       'scorePerMinute', 'distanceTraveled', 'teamSurvivalTime', 'deaths',
       'objectiveDestroyedEquipment', 'kdRatio', 'bonusXp', 'gulagDeaths',
       'timePlayed', 'executions', 'gulagKills', 'nearmisses',
       'objectiveBrCacheOpen', 'percentTimeMoving', 'miscXp', 'longestStreak',
       'teamPlacement', 'damageDone', 'damageTaken',
       'objectiveBrMissionPickupTablet', 'objectiveBrKioskBuy',
       'objectiveTeamWiped', 'objectiveLastStandKill',
       'objectiveBrDownEnemyCircle3', 'objectiveBrDownEnemyCircle1',
       'objectiveMunitionsBoxTeammateUsed', 'objectiveBrDownEnemyCircle4',
       'objectiveBrDownEnemyCircle2', 'objectiveReviver',
       'objectiveTrophyDefense', 'objectiveDestroyedVehicleMedium'],
      dtype='object')

In [12]:
player_stats[['headshots', 'distanceTraveled', 'teamSurvivalTime', 'objectiveBrKioskBuy']].head(5)

,headshots,distanceTraveled,teamSurvivalTime,objectiveBrKioskBuy
0,5.0,74418.586,794448.0,NaN
1,0.0,195506.890,439056.0,NaN
2,0.0,385011.300,1284144.0,2.0
3,0.0,390076.880,1295520.0,2.0
4,1.0,265620.470,822624.0,1.0


#### player col

In [13]:
player = raw['player'].apply(pd.Series)
player.head(5)

,team,rank,awards,username,uno,clantag,loadouts,loadout,brMissionStats
0,axis,54.0,{},gentil_renard,2621859779580650696,lkf :,"[{'primaryWeapon': {'name': 'iw8_ar_falima', '...","[{'primaryWeapon': {'name': 'iw8_ar_falima', '...",NaN
1,team_eighteen,54.0,{},gentil_renard,2621859779580650696,lkf :,"[{'primaryWeapon': {'name': 'iw8_ar_valpha', '...","[{'primaryWeapon': {'name': 'iw8_ar_valpha', '...","{'missionsComplete': 0, 'totalMissionXpEarned'..."
2,team_twenty_three,54.0,{},gentil_renard,2621859779580650696,lkf :,"[{'primaryWeapon': {'name': 'iw8_ar_valpha', '...","[{'primaryWeapon': {'name': 'iw8_ar_valpha', '...","{'missionsComplete': 2, 'totalMissionXpEarned'..."
3,team_fourteen,54.0,{},gentil_renard,2621859779580650696,lkf :,"[{'primaryWeapon': {'name': 'iw8_ar_falima', '...","[{'primaryWeapon': {'name': 'iw8_ar_falima', '...","{'missionsComplete': 1, 'totalMissionXpEarned'..."
4,team_eighteen,54.0,{},gentil_renard,2621859779580650696,lkf :,"[{'primaryWeapon': {'name': 'iw8_ar_valpha', '...","[{'primaryWeapon': {'name': 'iw8_ar_valpha', '...","{'missionsComplete': 1, 'totalMissionXpEarned'..."


In [14]:
player.keys()

Index(['team', 'rank', 'awards', 'username', 'uno', 'clantag', 'loadouts',
       'loadout', 'brMissionStats'],
      dtype='object')

#### Deep dive into player.loadout entry

In [15]:
# Each entry of 'loadout' is a list of dict. Either one dict (if 1 loadout) or more(2 or more loadouts taken)
# Example, matche at index 11 with two loadouts uses (one list with 2 dict).
player['loadout'][11]

[{'primaryWeapon': {'name': 'iw8_ar_valpha',
   'label': None,
   'imageLoot': None,
   'imageIcon': None,
   'variant': '0',
   'attachments': [{'name': 'xmags',
     'label': None,
     'image': None,
     'category': None},
    {'name': 'pistolgrip03', 'label': None, 'image': None, 'category': None},
    {'name': 'gripvertpro', 'label': None, 'image': None, 'category': None},
    {'name': 'laserrange', 'label': None, 'image': None, 'category': None},
    {'name': 'barcust', 'label': None, 'image': None, 'category': None}]},
  'secondaryWeapon': {'name': 'iw8_sn_t9accurate',
   'label': None,
   'imageLoot': None,
   'imageIcon': None,
   'variant': '0',
   'attachments': [{'name': 'stockcust',
     'label': None,
     'image': None,
     'category': None},
    {'name': 'laserrange', 'label': None, 'image': None, 'category': None},
    {'name': 'silencer2', 'label': None, 'image': None, 'category': None},
    {'name': 'barcust', 'label': None, 'image': None, 'category': None},
    {'

In [16]:
# first loadout structure 
player['loadout'][11][1].keys()

dict_keys(['primaryWeapon', 'secondaryWeapon', 'perks', 'extraPerks', 'killstreaks', 'tactical', 'lethal'])

In [17]:
player['loadout'][11][1]['primaryWeapon'].keys()

dict_keys(['name', 'label', 'imageLoot', 'imageIcon', 'variant', 'attachments'])

#### expanded data : raw + playerStats & player columns expanded + player.loadout expanded

In [18]:
expanded = pd.DataFrame(loaded_matches)
expanded = pd.concat([expanded.drop(['playerStats'], axis=1), expanded['playerStats'].apply(pd.Series)], axis=1)
expanded = pd.concat([expanded.drop(['player'], axis=1), expanded['player'].apply(pd.Series)], axis=1)
expanded = expanded.drop(['brMissionStats'], axis = 1)

# player.loadout column
expanded = pd.concat([expanded.drop(['loadout'], axis=1), expanded['loadout'].apply(pd.Series)], axis=1)
for col in range(0,3):
    if col in expanded.columns:
        expanded[col] = expanded[col].apply(lambda x: [x['primaryWeapon']['name'],x['secondaryWeapon']['name']] if not str(x) == 'nan' else np.nan)
        expanded = expanded.rename(columns={col: f"loadout_{str(col +1)}"})
expanded.head(15)

,utcStartSeconds,utcEndSeconds,map,mode,matchID,duration,playlistName,version,gameType,playerCount,...,team,rank,awards,username,uno,clantag,loadouts,loadout_1,loadout_2,loadout_3
0,1636049077,1636049872,mp_don4,br_rumble_clash,4779687662268956307,795000,None,1,wz,100,...,axis,54.0,{},gentil_renard,2621859779580650696,lkf :,"[{'primaryWeapon': {'name': 'iw8_ar_falima', '...","[iw8_ar_falima, iw8_sn_t9accurate]",NaN,NaN
1,1635974372,1635975965,mp_don4,br_brquads,3670970687107920841,1593000,None,1,wz,152,...,team_eighteen,54.0,{},gentil_renard,2621859779580650696,lkf :,"[{'primaryWeapon': {'name': 'iw8_ar_valpha', '...","[iw8_ar_valpha, iw8_sn_t9cannon]","[iw8_pi_t9semiauto, iw8_fists]",NaN
2,1635972898,1635974510,mp_don4,br_brquads,2776548585301277918,1612000,None,1,wz,152,...,team_twenty_three,54.0,{},gentil_renard,2621859779580650696,lkf :,"[{'primaryWeapon': {'name': 'iw8_ar_valpha', '...","[iw8_ar_valpha, iw8_sn_t9cannon]","[iw8_pi_t9semiauto, iw8_fists]",NaN
3,1635971331,1635972934,mp_don4,br_brquads,12283805539552672972,1603000,None,1,wz,151,...,team_fourteen,54.0,{},gentil_renard,2621859779580650696,lkf :,"[{'primaryWeapon': {'name': 'iw8_ar_falima', '...","[iw8_ar_falima, iw8_sn_t9accurate]",NaN,NaN
4,1635970345,1635971989,mp_don4,br_brquads,3648604903481562804,1644000,None,1,wz,152,...,team_eighteen,54.0,{},gentil_renard,2621859779580650696,lkf :,"[{'primaryWeapon': {'name': 'iw8_ar_valpha', '...","[iw8_ar_valpha, iw8_sn_t9cannon]","[iw8_pi_t9semiauto, iw8_fists]",NaN
5,1635890556,1635892107,mp_don4,br_brquads,9621351053666542885,1551000,None,1,wz,152,...,team_fifteen,54.0,{},gentil_renard,2621859779580650696,lkf :,"[{'primaryWeapon': {'name': 'iw8_ar_falima', '...","[iw8_ar_falima, iw8_sn_t9accurate]",NaN,NaN
6,1635889951,1635891573,mp_don4,br_brquads,15502873101340219395,1622000,None,1,wz,152,...,team_eighteen,54.0,{},gentil_renard,2621859779580650696,lkf :,"[{'primaryWeapon': {'name': 'iw8_ar_valpha', '...","[iw8_ar_valpha, iw8_sn_t9cannon]","[iw8_pi_t9semiauto, iw8_fists]",NaN
7,1635889282,1635890865,mp_don4,br_brquads,18320891797642958889,1583000,None,1,wz,152,...,team_nineteen,54.0,{},gentil_renard,2621859779580650696,lkf :,"[{'primaryWeapon': {'name': 'iw8_ar_valpha', '...","[iw8_ar_valpha, iw8_sn_t9cannon]","[iw8_pi_t9semiauto, iw8_fists]",NaN
8,1635888314,1635889897,mp_don4,br_brquads,16361016574177890628,1583000,None,1,wz,153,...,team_twenty_three,54.0,{},gentil_renard,2621859779580650696,lkf :,"[{'primaryWeapon': {'name': 'iw8_ar_falima', '...","[iw8_ar_falima, iw8_sn_t9accurate]","[iw8_pi_t9semiauto, iw8_fists]",NaN
9,1635887584,1635889102,mp_don4,br_brtrios,3439852252647514399,1518000,None,1,wz,150,...,team_eight,54.0,{},gentil_renard,2621859779580650696,lkf :,"[{'primaryWeapon': {'name': 'iw8_ar_valpha', '...","[iw8_ar_valpha, iw8_sn_t9cannon]","[iw8_pi_t9semiauto, iw8_fists]",NaN


#### Functions

In [19]:
def getLastMatchId(matches):
    """ Extract last (Battle Royale) Match ID from last matches """
    list_br_match = [match for match in matches if "br_br" in match["mode"]]
    last_match_id = int(list_br_match[0]["matchID"]) if len(list_br_match) > 0 else None
    return last_match_id


def MatchesToDf(matches):
    """
    COD API / callofduty.py client --> list of dict : max 20 Matches with Player stats
    We expand Player and PlayerStats levels, concatenate them
    ! We drop some cols we dont want to work with

    Returns
    -------
    DataFrame, matches as rows, matches/ one player stats as columns
    """
    
    
    keep_cols =  [
        'mode',
        'utcStartSeconds',
        'utcEndSeconds',
        'timePlayed',
        'teamPlacement',
        'kdRatio', 
        'kills', 
        'deaths', 
        'assists', 
        'damageDone',
        'damageTaken',
        'gulagKills',
        'percentTimeMoving',
        'duration'
        ]

    df = pd.DataFrame(matches)
    
    # column playerStats is a series of dict, we can expand it easily and append, then drop the original
    df = pd.concat([df.drop(['playerStats'], axis=1), df['playerStats'].apply(pd.Series)], axis=1)
    
    # colum player has more depth
    # once expanded, it has a column 'loadout' : a series of list of dict (either one or more, we will keep 1 to max 3)
    # and also brMissionStats (mostly empty ?) that we aren't interested in
    
    df = pd.concat([df.drop(['player'], axis=1), df['player'].apply(pd.Series)], axis=1)
    df = df.drop(['brMissionStats'], axis = 1)
    df = pd.concat([df.drop(['loadout'], axis=1), df['loadout'].apply(pd.Series)], axis=1)
    for col in range(0,3):
        if col in df.columns:
            df[col] = df[col].apply(lambda x: f"{x['primaryWeapon']['name']} - {x['secondaryWeapon']['name']}" if not str(x) == 'nan' else np.nan)
            col_name = "loadout_" + str(col +1)
            df = df.rename(columns={col: f"loadout_{str(col +1)}"})   
            keep_cols.append(col_name)
    
    return df[keep_cols]

In [20]:
df = MatchesToDf(loaded_matches)
df.head(3)

,mode,utcStartSeconds,utcEndSeconds,timePlayed,teamPlacement,kdRatio,kills,deaths,assists,damageDone,damageTaken,gulagKills,percentTimeMoving,duration,loadout_1,loadout_2,loadout_3
0,br_rumble_clash,1636049077,1636049872,896.0,2.0,1.5,12.0,8.0,4.0,2925.0,823.0,0.0,91.14249,795000,iw8_ar_falima - iw8_sn_t9accurate,NaN,NaN
1,br_brquads,1635974372,1635975965,515.0,30.0,0.0,0.0,2.0,0.0,26.0,326.0,0.0,66.36155,1593000,iw8_ar_valpha - iw8_sn_t9cannon,iw8_pi_t9semiauto - iw8_fists,NaN
2,br_brquads,1635972898,1635974510,1371.0,11.0,0.0,0.0,2.0,0.0,266.0,475.0,0.0,74.08844,1612000,iw8_ar_valpha - iw8_sn_t9cannon,iw8_pi_t9semiauto - iw8_fists,NaN


In [21]:
last_match_id = getLastMatchId(loaded_matches)
last_match_id

3670970687107920841

In [24]:
def MatchesStandardize(df):
    """
    A first layer of standadization (as : properly formated) to our matches DataFrame
    For further aggregations / better readibility of our data
    
    Returns
    -------
    DataFrame : matches as rows, cleaned matches/player stats as columns
    """
    
    int_cols =  [
        'teamPlacement', 
        'kills', 
        'deaths', 
        'assists', 
        'gulagKills', 
        'damageDone',
        'damageTaken'
        ]
    
    float_cols = [
        'kdRatio',
        'percentTimeMoving'
        ]
    
    ts_cols = [
        'utcStartSeconds',
        'utcEndSeconds'
        ]
    
    columns_labels = {
        'utcEndSeconds':'Ended at',
        'utcStartSeconds':'Started at',
        'timePlayed': 'Playtime',
        'teamPlacement':'#',
        'kdRatio':'KD',
        'damageDone':'Damage >',
        'damageTaken':'Damage <',
        'gulagKills':'Gulag',
        'headshots':'% headshots',
        'percentTimeMoving':'% moving',
        'duration':'Game duration'
        }

    df = df.fillna(0) # sometimes (Plunder only ?) rank, gulag is NaN
    df[int_cols] = df[int_cols].astype(int)
    df[float_cols] = df[float_cols].astype(float).round(1) # still renders 0.0000 in streamlit but ugly hacks exists
    
    # specials
    df['utcEndSeconds'] = df['utcEndSeconds'].apply(lambda x: datetime.fromtimestamp(x))
    df['utcStartSeconds'] = df['utcStartSeconds'].apply(lambda x: datetime.fromtimestamp(x))
    
    df['duration'] = df['duration'].apply(lambda x: x/1000).apply(lambda x: pd.to_datetime(x, unit='s').strftime('%M')) # API duration is in seconds x1000
    df['timePlayed'] = df['timePlayed'].apply(lambda x: pd.to_datetime(x, unit='s').strftime('%M:%S')) # API timePlayed is in seconds
    df['gulagKills'] = df['gulagKills'].map({1:'W', 0:'L'})
    for col in ['loadout_1', 'loadout_2', 'loadout_3']:
            df.fillna({col:'-'}, inplace=True) if col in df.columns else None
    
    df = df.replace({"mode": mode_labels})
    df = df.rename(columns=columns_labels)
    df.columns = df.columns.str.capitalize()
    df = df.rename({"Kd":"KD"}, axis=1)
     
    return df

In [25]:
raw = pd.DataFrame(loaded_matches)
df = MatchesToDf(raw)
df = MatchesStandardize(df)
df.head(5)

,Mode,Started at,Ended at,Playtime,#,KD,Kills,Deaths,Assists,Damage >,Damage <,Gulag,% moving,Game duration,Loadout_1,Loadout_2,Loadout_3
0,Rumble,2021-11-04 19:04:37,2021-11-04 19:17:52,14:56,2,1.5,12,8,4,2925,823,L,91.1,13,iw8_ar_falima - iw8_sn_t9accurate,0,0
1,Quads,2021-11-03 22:19:32,2021-11-03 22:46:05,08:35,30,0.0,0,2,0,26,326,L,66.4,26,iw8_ar_valpha - iw8_sn_t9cannon,iw8_pi_t9semiauto - iw8_fists,0
2,Quads,2021-11-03 21:54:58,2021-11-03 22:21:50,22:51,11,0.0,0,2,0,266,475,L,74.1,26,iw8_ar_valpha - iw8_sn_t9cannon,iw8_pi_t9semiauto - iw8_fists,0
3,Quads,2021-11-03 21:28:51,2021-11-03 21:55:34,22:46,7,2.0,2,1,4,1145,100,L,96.7,26,iw8_ar_falima - iw8_sn_t9accurate,0,0
4,Quads,2021-11-03 21:12:25,2021-11-03 21:39:49,15:01,23,2.0,4,2,0,1026,374,W,84.8,27,iw8_ar_valpha - iw8_sn_t9cannon,iw8_pi_t9semiauto - iw8_fists,0


In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 17 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Mode           20 non-null     object        
 1   Started at     20 non-null     datetime64[ns]
 2   Ended at       20 non-null     datetime64[ns]
 3   Playtime       20 non-null     object        
 4   #              20 non-null     int64         
 5   KD             20 non-null     float64       
 6   Kills          20 non-null     int64         
 7   Deaths         20 non-null     int64         
 8   Assists        20 non-null     int64         
 9   Damage >       20 non-null     int64         
 10  Damage <       20 non-null     int64         
 11  Gulag          20 non-null     object        
 12  % moving       20 non-null     float64       
 13  Game duration  20 non-null     object        
 14  Loadout_1      20 non-null     object        
 15  Loadout_2      20 non-nul

In [27]:
loadout_cols = df.columns[df.columns.str.startswith('Loadout')].tolist()
loadout_cols

['Loadout_1', 'Loadout_2', 'Loadout_3']

In [28]:
def MatchesPerDay(df):
    """
    Final layer applied to our list of matches with stats, to render them in our Streamlit App
    Streamlit/basic AgGrid does not render well (aka w. blank rows etc.) multi indexed df
    So we structure and display our data differently (a dictionary instead of a df), in a daily manner
    
    Returns
    -------
    Dictionary :
    {
        "str_weekday_1":df-of-matches-that-day,
        "str_weekday_2":df-of-matches-that-day,
        (...)
    }
    """
    
    drop_cols =  [
        'Started at',
        'Playtime',
        '% moving',
        'Game duration'
        ]
    
    keep_cols = [
        'End time',
        'Mode',
        '#',
        'KD',
        'Kills',
        'Deaths',
        'Assists',
        'Damage >',
        'Damage <',
        'Gulag'
    ]
    
    loadout_cols = df.columns[df.columns.str.startswith('Loadout')].tolist()
    df[loadout_cols] = df[loadout_cols].replace(0, '-') # else can't concat Loadouts cols
    
    # 1. --- initial formating : datetime, concat loadouts cols ---
    
    df = df.drop(drop_cols, axis = 1)
    df['End time'] = df['Ended at'].dt.time
    
    def concat_loadouts(df, columns):
        return pd.Series(map(' , '.join, df[columns].values.tolist()),index = df.index)
    
    df['Weapons'] = concat_loadouts(df, loadout_cols)
    keep_cols = [*keep_cols, *['Weapons']]
    
    # 2. --- build result {"day1": df-matches-that-day, "day2": df...} ---
    
    list_df = [g for n, g in df.groupby(pd.Grouper(key='Ended at',freq='D'))]
    list_df = [df for df in list_df if not df.empty]
    list_days = [df['Ended at'].tolist()[0].strftime('%A') for df in list_df]
    
    # make sure we display latest day first, then build dictionary
    for list_ in [list_days, list_df]:
        list_.reverse()
    day_matches = dict(zip(list_days, list_df))
    
    # 3. --- some more (re)formating ---
    
    # for some reason (me ? ^-^, Grouper => Series?) couldnt' modify df before building the result, must iterate again 
    for k, v in day_matches.items():
        day_matches[k] = day_matches[k][keep_cols]
      
    return day_matches


def AggStats(df):
    """
    We want to add aggregated stats for each day/df of matches we got from MatchesPerDay()
    Each daily aggregation will be rendered in our Streamlit App on top of each list of matches
    
    Returns
    -------
    Dictionary :
    {
    'Kills':total n kills that day,
    'Deaths': total deaths,
    'KD': kills/deaths,
    'Gulags': win % ,
    'Played': count matches -
    }
    """

    agg_func = {
        "Mode":"count",
        "Kills":"sum",
        "Deaths":"sum"
    } 
    dict_ = df.agg(agg_func).to_dict()
    dict_.update({'KD': (df.Kills.sum() / df.Deaths.sum()).round(2)})
    dict_.update({'Gulags': int((df.Gulag.str.count("W").sum() / df.Gulag.str.count("L").sum())*100)})
    dict_['Played'] = dict_.pop('Mode')
    
    
    return dict_

In [29]:
raw = pd.DataFrame(loaded_matches)
df = MatchesToDf(raw)
df = MatchesStandardize(df)
daily = MatchesPerDay(df)

In [30]:
for day in daily.keys():
    print(day)
    display(daily[day])
    print(daily[day]['Mode'].values)

Thursday


,End time,Mode,#,KD,Kills,Deaths,Assists,Damage >,Damage <,Gulag,Weapons
0,19:17:52,Rumble,2,1.5,12,8,4,2925,823,L,"iw8_ar_falima - iw8_sn_t9accurate , - , -"


['Rumble']
Wednesday


,End time,Mode,#,KD,Kills,Deaths,Assists,Damage >,Damage <,Gulag,Weapons
4,21:39:49,Quads,23,2.0,4,2,0,1026,374,W,"iw8_ar_valpha - iw8_sn_t9cannon , iw8_pi_t9sem..."
3,21:55:34,Quads,7,2.0,2,1,4,1145,100,L,"iw8_ar_falima - iw8_sn_t9accurate , - , -"
2,22:21:50,Quads,11,0.0,0,2,0,266,475,L,"iw8_ar_valpha - iw8_sn_t9cannon , iw8_pi_t9sem..."
1,22:46:05,Quads,30,0.0,0,2,0,26,326,L,"iw8_ar_valpha - iw8_sn_t9cannon , iw8_pi_t9sem..."


['Quads' 'Quads' 'Quads' 'Quads']
Tuesday


,End time,Mode,#,KD,Kills,Deaths,Assists,Damage >,Damage <,Gulag,Weapons
19,18:58:25,Pldr x3,0,2.1,15,7,2,3456,895,L,"iw8_sh_dpapa12 - iw8_sn_t9powersemi , iw8_sh_d..."
18,19:22:45,Duos,31,1.5,3,2,1,1049,660,W,"iw8_ar_valpha - iw8_sn_t9cannon , iw8_pi_t9sem..."
17,19:39:02,Duos,58,0.0,0,2,0,103,200,L,"iw8_ar_valpha - iw8_sn_t9cannon , iw8_pi_t9sem..."
16,19:47:20,Duos,16,0.5,1,2,0,1454,686,L,"iw8_ar_valpha - iw8_sn_t9cannon , iw8_pi_t9sem..."
15,20:05:04,Duos,66,0.5,1,2,0,102,391,W,"iw8_ar_valpha - iw8_sn_t9cannon , iw8_pi_t9sem..."
14,20:13:36,Duos,29,1.5,3,2,0,791,624,W,"iw8_ar_valpha - iw8_sn_t9cannon , iw8_pi_t9sem..."
13,20:30:01,Trios,27,1.0,2,2,2,892,559,W,"iw8_ar_valpha - iw8_sn_t9cannon , iw8_pi_t9sem..."
12,20:41:34,Trios,32,1.5,3,2,1,1412,332,W,"iw8_sh_dpapa12 - iw8_sn_t9powersemi , iw8_pi_t..."
11,22:10:36,Trios,28,0.5,1,2,1,572,843,W,"iw8_ar_valpha - iw8_sn_t9accurate , iw8_pi_t9s..."
10,22:24:30,Trios,21,0.0,0,3,0,112,460,L,"iw8_ar_valpha - iw8_sn_t9cannon , iw8_pi_t9sem..."


['Pldr x3' 'Duos' 'Duos' 'Duos' 'Duos' 'Duos' 'Trios' 'Trios' 'Trios'
 'Trios' 'Trios' 'Quads' 'Quads' 'Quads' 'Quads']


In [ ]:
for k, v in daily.items():
    print(k)
    print(AggStats(v))

In [ ]:
df = daily["Thursday"].copy(deep=True)
df

In [ ]:
list_br_modes = ['Duos', 'Trios', 'Quads', 'Iron Trials', 'xxxx']
df[df['Mode'].isin(list_br_modes)]

In [ ]:
df['Mode'].unique().tolist()